# HW05 - Taming Text

## 1. Generate a word cloud based on the raw corpus

In [1]:
import numpy as np
import pandas as pd
import nltk

Getting the data

In [16]:
emails=pd.DataFrame.from_csv('./hillary-clinton-emails/Emails.csv')
emails.head(3)

,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,MetadataDocumentClass,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
Id,,,,,,,,,,,,,,,,,,,,,
1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,HRC_Email_296,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,HRC_Email_296,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,HRC_Email_296,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...


In [33]:
for column in emails.columns:
    print(column, "\n", emails[column].head(3), "\n")

DocNumber 
 Id
1    C05739545
2    C05739546
3    C05739547
Name: DocNumber, dtype: object 

MetadataSubject 
 Id
1                                                  WOW
2    H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...
3                                        CHRIS STEVENS
Name: MetadataSubject, dtype: object 

MetadataTo 
 Id
1     H
2     H
3    ;H
Name: MetadataTo, dtype: object 

MetadataFrom 
 Id
1    Sullivan, Jacob J
2                  NaN
3      Mills, Cheryl D
Name: MetadataFrom, dtype: object 

SenderPersonId 
 Id
1    87.0
2     NaN
3    32.0
Name: SenderPersonId, dtype: float64 

MetadataDateSent 
 Id
1    2012-09-12T04:00:00+00:00
2    2011-03-03T05:00:00+00:00
3    2012-09-12T04:00:00+00:00
Name: MetadataDateSent, dtype: object 

MetadataDateReleased 
 Id
1    2015-05-22T04:00:00+00:00
2    2015-05-22T04:00:00+00:00
3    2015-05-22T04:00:00+00:00
Name: MetadataDateReleased, dtype: object 

MetadataPdfLink 
 Id
1    DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...
2 

In [29]:
[x for x in emails.RawText[:3]]

['UNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.\nRELEASE IN FULL\nFrom: Sullivan, Jacob J <Sullivan11@state.gov>\nSent: Wednesday, September 12, 2012 10:16 AM\nTo:\nSubject: FW: Wow\nFrom: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi\nSent: Wednesday, September 12, 2012 10:09 AM\nTo: Sullivan, Jacob J\nSubject: Wow\nWhat a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me\ntoo\nUNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739545\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247\n\x0c',
 "UNCLASSIFIED\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739

## Using Python word_cloud library

## Using Natural Language Toolkit

### Pre-processing